In [5]:
import glob
import numpy as np
import pandas as pd
import csv
import tensorflow as tf
from keras.layers import Dense, Activation, Flatten, Dropout, LSTM, Conv1D
from keras.models import Sequential, Model
from keras import regularizers, optimizers
import keras
import sklearn

In [2]:
cough_df = pd.read_csv("cough_dataset.csv")
del cough_df['filename']
cough_df['label']="Cough"
no_cough_df = pd.read_csv("nocough_dataset.csv")
del no_cough_df['filename']
no_cough_df['label']="No Cough"
no_cough_df = no_cough_df.sample(frac=1) #Shuffle dataset to ensure no-cough 
no_cough_df = no_cough_df.head(len(cough_df)) #The two datasets should be the same length 

In [3]:
full_df = pd.concat([cough_df,no_cough_df])
features = full_df.columns[0:-1]
labels = pd.get_dummies(full_df['label']).values
"""
I chose to one-hot-encode the labels; originally I had assigned 0 to Covid and 1 to No Covid, but this made interpreting results incredibly difficult.
pd.get_dummies() turns the results into two columns - if the original value was "Covid," the row is [1,0]. Otherwise, the value is [0,1].
Intepreting model predictions becomes much easier; if a higher value is in the first column, the model has predicted Cough, otherwise, the model has predicted No Cough
"""
input_data = full_df[features].values

In [4]:
X_train, X_test, y_train, y_test =  sklearn.model_selection.train_test_split(input_data,labels,test_size=0.1)

In [5]:
"""
There isn't a whole lot of logic to this so far - the model needs improving. 
"""

def get_basic_model():
  model = Sequential()
  model.add(Conv1D(124, 33, padding='same',input_shape=(56,1),activation='relu'))
  model.add(Dense(106,kernel_regularizer='l1'))
  model.add(Dropout(.2))
  model.add(LSTM(105,return_sequences=True))
  model.add(Dropout(.2))
  model.add(Flatten())
  model.add(Dense(2,activation="sigmoid")) #Output has two columns (Covid, Non-Covid). Sigmoid activation is also supposed to be best for class-based entropy

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  model.summary()
  return model

In [6]:
model = get_basic_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 56, 124)           4216      
                                                                 
 dense (Dense)               (None, 56, 106)           13250     
                                                                 
 dropout (Dropout)           (None, 56, 106)           0         
                                                                 
 lstm (LSTM)                 (None, 56, 105)           89040     
                                                                 
 dropout_1 (Dropout)         (None, 56, 105)           0         
                                                                 
 flatten (Flatten)           (None, 5880)              0         
                                                                 
 dense_1 (Dense)             (None, 2)                 1

In [9]:
model.fit(X_train,y_train,epochs=20,validation_split=0.1)

Epoch 1/20
304/304 [==============================] - 21s 61ms/step - loss: 0.8351 - accuracy: 0.7614 - precision: 0.7616 - recall: 0.7613 - val_loss: 0.7783 - val_accuracy: 0.7630 - val_precision: 0.7664 - val_recall: 0.7472
Epoch 2/20
304/304 [==============================] - 18s 60ms/step - loss: 0.8014 - accuracy: 0.7681 - precision: 0.7674 - recall: 0.7690 - val_loss: 0.7219 - val_accuracy: 0.7676 - val_precision: 0.7666 - val_recall: 0.7694
Epoch 3/20
304/304 [==============================] - 19s 61ms/step - loss: 0.8165 - accuracy: 0.7719 - precision: 0.7718 - recall: 0.7715 - val_loss: 0.6954 - val_accuracy: 0.7583 - val_precision: 0.7588 - val_recall: 0.7574
Epoch 4/20
304/304 [==============================] - 15s 49ms/step - loss: 0.6217 - accuracy: 0.7819 - precision: 0.7812 - recall: 0.7826 - val_loss: 0.6965 - val_accuracy: 0.7620 - val_precision: 0.7608 - val_recall: 0.7685
Epoch 5/20
304/304 [==============================] - 13s 44ms/step - loss: 0.8304 - accuracy: 0

In [8]:
"""
Want to make sure that predictions are correct! More importantly, want to make sure that coughs are correctly classified as coughs. 
Overall accuracy isn't the best metric - we care more about f-score (precision/recall). 
Precision: What was correctly classified as cough?  True Coughs/(True Coughs + False Coughs)
Recall: Of what was actually a cough, what percentage was correctly classified?  True Coughs/(True Coughs + False Non-Coughs )
"""

predictions = model.predict(X_test)
reals = []
preds = []
for i in range(len(predictions)):
    reals.append(np.argmax(y_test[i]))
    preds.append(np.argmax(predictions[i]))
sklearn.metrics.precision_recall_fscore_support(reals,preds) #First column is cough, second column is non-cough

38/38 [==============================] - 1s 20ms/step


(array([0.71940299, 0.78867925]),
 array([0.81144781, 0.68976898]),
 array([0.76265823, 0.73591549]),
 array([594, 606], dtype=int64))